## Библиотеки 

In [3]:
import numpy as np
import pandas as pd
from collections import Counter

import warnings # ?
warnings.filterwarnings("ignore") # убираем оповещения о предупреждениях. Не засоряем вывод
print('Let me Die')

Let me Die


## Получение данных

In [4]:
df_train = pd.read_csv('F:\Anac_python\Examples_Python\Jupyter_Notebook_stuff\Alpha_test\alfabank-campus\df_train.csv', sep=';')
df_test = pd.read_csv('F:\Anac_python\Examples_Python\Jupyter_Notebook_stuff\Alpha_test\alfabank-campus\df_test.csv', sep=';')

OSError: [Errno 22] Invalid argument: 'F:\\Anac_python\\Examples_Python\\Jupyter_Notebook_stuff\\Alpha_test\x07lfabank-campus\\df_train.csv'

In [ ]:
df_train['Data'] = df_train.Data.apply(lambda s: list(map(int, s.split(','))))
df_train['Target'] = df_train.Target.apply(lambda s: list(map(int, s.split(','))))
df_test['Data'] = df_test.Data.apply(lambda s: list(map(int, s.split(','))))

In [ ]:
df_train.head()

In [ ]:
df_train.shape

In [ ]:
df_train.dtypes

In [ ]:
df_test.head()

In [ ]:
df_test.shape

In [ ]:
df_test.dtypes

## Функции (чё?)

In [ ]:
# https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.

    This function computes the average prescision at k between two lists of
    items.

    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The average precision at k over the input lists

    """
    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])

## Baseline 1: топ10 MCC-кодов из train-части

MCC-код и соответствующее количество вхождений в train-часть

In [ ]:
top10_codes = df_train['Data'].explode().value_counts().head(10)
top10_codes

In [ ]:
mapk(df_train['Target'], [top10_codes.index]*len(df_train))

## Baseline 2: cамые популярные транзакции пользователя.

**Если таких менее 10, то замешиваются топ10 популярных из всей выборки.**

In [ ]:
def get_top_codes(transactions, top_n=10, drop_from=5):
    transactions_stats = sorted(
        Counter(transactions).items(), 
        key=lambda x: x[1], 
        reverse=True
    )[:top_n]
    

    top_codes = [mcc_code for (mcc_code, count) in transactions_stats if count >= drop_from]
    top_codes += list(top10_codes.index)

    return top_codes[:10]

In [ ]:
df_train['pred_baseline_2'] = df_train['Data'].apply(get_top_codes)

In [ ]:
mapk(df_train['Target'], df_train['pred_baseline_2'])

## Submission

предсказания из второго бейзлайна

In [ ]:
df_test['Predicted'] = df_test['Data'].apply(get_top_codes)

In [ ]:
df_test.head()

In [ ]:
submission_baseline_2 = df_test[['Id', 'Predicted']]
submission_baseline_2['Predicted'] = submission_baseline_2.Predicted.astype(str).str.replace(',', '')
submission_baseline_2.to_csv('submission_baseline_2.csv', index=False)